# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
CityWeather=pd.read_csv('CityWeather.csv') # Importing Cities CSV exported earlier
CityWeather.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,polunochnoye,60.8697,60.4153,33.53,79,20,8.61,RU,1634669089
1,berlevag,70.8578,29.0864,28.35,80,40,12.66,NO,1634669089
2,rikitea,-23.1203,-134.9692,74.93,71,0,20.42,PF,1634669090
3,new norfolk,-42.7826,147.0587,39.70,74,6,1.92,AU,1634669090
4,hambantota,6.1241,81.1185,77.34,100,0,2.30,LK,1634669090


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations=CityWeather[['Lat','Lng']]
fig=gmaps.figure()
heat_layer=gmaps.heatmap_layer(locations,weights=CityWeather['Humidity'],
                              dissipating=False,max_intensity=100,point_radius=1.5) 
# Max Intensity set as 100, as max humidity is 100
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Finding Cities with max temperature lower than 80 degrees but higher than 70
#Wind speed less than 10 mph and 0 cloudiness
hotel_df=CityWeather[(CityWeather['Max Temp']<80)&(CityWeather['Max Temp']>70)&
                     (CityWeather['Wind Speed']<10)&(CityWeather['Cloudiness']==0)]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,hambantota,6.1241,81.1185,77.34,100,0,2.30,LK,1634669090
50,jati,24.3539,68.2671,78.62,61,0,9.69,PK,1634669107
95,jand,33.4299,72.0193,76.98,37,0,2.64,PK,1634669126
165,avera,33.1940,-82.5271,76.71,31,0,2.73,US,1634669147
168,daska,32.3243,74.3497,73.38,38,0,1.43,PK,1634669148
211,casablanca,33.5928,-7.6192,75.42,64,0,3.00,MA,1634668968
216,lujan,-34.5703,-59.1050,79.03,34,0,1.99,AR,1634669164
242,nueve de julio,-35.4444,-60.8831,73.71,35,0,5.61,AR,1634669042
308,vila franca do campo,37.7167,-25.4333,70.21,71,0,5.73,PT,1634669191
324,dobson,36.3957,-80.7226,75.00,36,0,3.44,US,1634669196


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
pd.set_option('mode.chained_assignment', None) #Set this option to supress warning.
hotel_df['Hotel Name']='' # Add Hotel Name column

In [7]:
base_url='https://maps.googleapis.com/maps/api/place/nearbysearch/json'
for index,row in hotel_df.iterrows():
    location= f"{row['Lat']},{row['Lng']}"
    params = {
    "location": location,
    #"keyword":"hotel", # If we use the keywork hotel, then 3 cities do not find a hotel.
    "radius":5000,
    "type": "lodging", #Using lodging type only 2 cities do not find a hotel.
    "key": g_key,
            }
    try:
        response=requests.get(base_url,params=params).json()
        hotel_df.loc[index,'Hotel Name']=response['results'][0]['name']
    except:
        print(f"No hotel found in {row['City']}. {response['status']}")

No hotel found in avera. ZERO_RESULTS
No hotel found in dobson. ZERO_RESULTS


In [8]:
# Remove citeis with no hotels.
hotel_df=hotel_df[hotel_df['Hotel Name']!='']
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
4,hambantota,6.1241,81.1185,77.34,100,0,2.30,LK,1634669090,Bungalow 63
50,jati,24.3539,68.2671,78.62,61,0,9.69,PK,1634669107,Cafe Lala Rehan Restaurant
95,jand,33.4299,72.0193,76.98,37,0,2.64,PK,1634669126,Mohalla Takia
168,daska,32.3243,74.3497,73.38,38,0,1.43,PK,1634669148,Royal Dream Fort
211,casablanca,33.5928,-7.6192,75.42,64,0,3.00,MA,1634668968,فندق البيت المغربي كازبلانكا
216,lujan,-34.5703,-59.1050,79.03,34,0,1.99,AR,1634669164,Hotel Biarritz
242,nueve de julio,-35.4444,-60.8831,73.71,35,0,5.61,AR,1634669042,Grand Hotel Libertad - GHL
308,vila franca do campo,37.7167,-25.4333,70.21,71,0,5.73,PT,1634669191,Pestana Bahia Praia
370,bhikhi,30.0667,75.5333,74.14,39,0,1.90,IN,1634669209,ਡਾਕਟਰ ਸਰੇਸ ਬਾਂਸਲ।
452,jodhpur,26.2867,73.0300,79.95,26,0,0.96,IN,1634669234,"Umaid Bhawan Palace, Jodhpur"


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map

locs=CityWeather[['Lat','Lng']] #locations for heatmap
fig=gmaps.figure()
#For all the cities in CSV create a heatmap of Humidity
heat_layer=gmaps.heatmap_layer(locs,weights=CityWeather['Humidity'],
                              dissipating=False,max_intensity=100,point_radius=1.5) 
#add heatmap to the gmaps
fig.add_layer(heat_layer)

#create a marker layer based on hotel_df
hotel_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)

#Add marker layer to gmaps.
fig.add_layer(hotel_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))